In [3]:
import pandas as pd
import numpy as np
import sqlite3

Таблица **application**:

- customer_id - общий ID клиена
- APP_ID - ID оффера (предложения о кредите клиенту)
- APP_DT - время оффера
- FULL_CREDIT_SUM - сумма кредита
- PRODUCT - продукт
- EDUCATION - образование клиента
- SEX - пол
- CREDIT_STATUS - статус заявки (оффера)


Таблица **customers_email**:

- customer_id - общий ID клиена
- EMAIL - email клиента 
- APP_DT - время оффера
- SEGMENT - сегмент
- date_fact - дата отправки email клиенту
- date_open - дата открытия email клиентом
- OPENING_COUNT - кол-во открытий email
- date_cross - дата перехода по ссылке в email
- CROSSING_COUNT - кол-во переходов по ссылке в email
- cnt_post - флаг отправки email
- cnt_del - флаг доставки email
- cnt_open - флаг открытия email
- cnt_cross - флаг перехода по ссылке в email

In [4]:
filename = "C:/Users/User/КУРС НОВЫЙ/data/email/Email"
con = sqlite3.connect(filename + ".db")


In [5]:
sql = '''select * from customers_email'''
customers_email = pd.read_sql(sql, con)

In [6]:
customers_email[:5]

,CUSTOMER_ID,EMAIL,SEGMENT,date_fact,date_open,OPENING_COUNT,date_cross,CROSSING_COUNT,cnt_post,cnt_del,cnt_open,cnt_cross
0,1,1@mail.ru,no_dbo,2020-03-20 00:00:00.0000000,None,NaN,None,NaN,1,0,0,0
1,1,1@mail.ru,no_dbo,2020-03-20 00:00:00.0000000,None,NaN,None,NaN,1,0,0,0
2,2,2@mail.ru,no_dbo,2020-03-20 00:00:00.0000000,2020-03-20 00:00:00.0000000,1.0,None,NaN,1,1,1,0
3,3,3@mail.ru,no_dbo,2020-03-20 00:00:00.0000000,None,NaN,None,NaN,1,1,0,0
4,3,3@mail.ru,no_dbo,2020-03-20 00:00:00.0000000,None,NaN,None,NaN,1,1,0,0


In [7]:
sql = '''select * from application'''
application = pd.read_sql(sql, con)

In [8]:
application[:5]

,index,customer_id,APP_ID,APP_DT,FULL_CREDIT_SUM,PRODUCT,EDUCATION,SEX,CREDIT_STATUS
0,0,4149,1,2020-03-17 00:00:00.0000000,999990,T0,Высшее,Женский,Rejected
1,1,685,2,2020-03-11 00:00:00.0000000,"439169,14",5L,Среднее,Мужской,Rejected
2,2,806,3,2020-03-12 00:00:00.0000000,245000,8F,Среднее специальное,Мужской,Financed
3,3,2711,4,2020-03-13 00:00:00.0000000,"809523,81",3W,Среднее специальное,Мужской,Client refused(approved)
4,4,4464,5,2020-03-19 00:00:00.0000000,1500000,T0,Высшее,Мужской,Rejected


1) Создать таблицу customer_unique на основе customers_email и пронумеровать при помощи оконной функции параметр customer_id отсортировав по полю date_fact в порядке убывания даты отправки email

In [9]:
cur = con.cursor()

In [10]:
sql = '''
        drop table if exists customer_unique;
        create table customer_unique as
        select distinct customer_id, date_fact,
        ROW_NUMBER() over(PARTITION BY customer_id order by date_fact) as rn
        from customers_email
        '''
cur.executescript(sql)

2) Вывести результат первого запроса (задания) при помощи pandas.read_sql

In [11]:
sql = '''select * from customer_unique'''
customer_unique = pd.read_sql(sql, con)
customer_unique[:5]

,CUSTOMER_ID,date_fact,rn
0,1,2020-03-20 00:00:00.0000000,1
1,1,2020-03-20 00:00:00.0000000,2
2,2,2020-03-20 00:00:00.0000000,1
3,3,2020-03-20 00:00:00.0000000,1
4,3,2020-03-20 00:00:00.0000000,2


3) Аналогично заданию 1, создать таблицу application_unique на основе customers_email и пронумеровать при помощи оконной функции параметр customer_id отсортировав по полю APP_DT

In [14]:
sql = '''
        drop table if exists application_unique;
        create table application_unique as
        select distinct c.customer_id, date_fact,
        ROW_NUMBER() over(PARTITION BY c.customer_id order by a.APP_DT) as rn
        from customers_email c
        inner join application a on c.customer_id = a.customer_id
        '''
cur.executescript(sql)

4) Вывести результат третьего задания при помощи pandas.read_sql

In [15]:
sql = '''select * from application_unique'''
application_unique = pd.read_sql(sql, con)
application_unique[:5]

,CUSTOMER_ID,date_fact,rn
0,7,2020-03-20 00:00:00.0000000,1
1,8,2020-03-20 00:00:00.0000000,1
2,9,2020-03-20 00:00:00.0000000,1
3,9,2020-03-20 00:00:00.0000000,2
4,9,2020-03-20 00:00:00.0000000,3


5) Написать запрос, который бы находил вторую по счету максимальную сумму кредита из application. Какое будет значение?

In [159]:
sql = '''
        select customer_id, cast(FULL_CREDIT_SUM as real)
        from application 
        order by cast(FULL_CREDIT_SUM as real) desc 
        limit 1,1
        '''
application_max = pd.read_sql(sql, con)
application_max[:5]

,customer_id,cast(FULL_CREDIT_SUM as real)
0,4560,3000000.0


6) Написать запрос, который бы находил уникальных клиентов с суммой запрашиваемого оффера больше, чем среднее значение суммы офера всех клиентов

In [132]:
sql = '''
        select customer_id, cast(FULL_CREDIT_SUM as real)
        from application 
        group by customer_id
        having cast(FULL_CREDIT_SUM as real)>(select cast(avg(FULL_CREDIT_SUM) as real) from application)
        '''
        
customer = pd.read_sql(sql, con)
customer[:5]

,customer_id,cast(FULL_CREDIT_SUM as real)
0,33,790000.0
1,42,557134.0
2,59,1000000.0
3,70,593471.0
4,107,636000.0


7) Напишите запрос, который выводит максимальную сумму для каждого уникального клиента при статусе отказа оффера credit_status like '%Reject%' (подсказка: использовать application)

In [227]:
sql = '''
        SELECT distinct a.customer_id, a.FULL_CREDIT_SUM, a.credit_status
        FROM (SELECT customer_id, FULL_CREDIT_SUM, credit_status,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY FULL_CREDIT_SUM DESC) rank
        FROM application) a
        WHERE a.rank = 1 and credit_status LIKE '%Reject%'
        '''
        
credit_status = pd.read_sql(sql, con)
credit_status[:5]

,customer_id,FULL_CREDIT_SUM,credit_status
0,8,60000,Rejected
1,9,"228342,19",Rejected
2,13,"568504,83",Rejected
3,26,"12128,56",Rejected
4,33,800000,Rejected


8) Вывеси максимальную сумму кредита для каждого клиента в зависимости от статуса оффера (подсказка: group by)

In [251]:
sql = '''
        SELECT a.customer_id, a.FULL_CREDIT_SUM, a.credit_status
        FROM (SELECT customer_id, FULL_CREDIT_SUM, credit_status,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY FULL_CREDIT_SUM DESC) rank
        FROM application) a
        WHERE a.rank = 1
        group by a.CREDIT_STATUS
        '''
        
credit_status2 = pd.read_sql(sql, con)
credit_status2[:30]

,customer_id,FULL_CREDIT_SUM,credit_status
0,208,"499147,24",Approved
1,59,1000000,Client refused(approved)
2,139,600000,Client refused(not finalized)
3,7,27000,Financed
4,634,750000,Processing
5,8,60000,Rejected


9) Вывести из application тех клиентов, чьих офферов в application больше, чем 2

In [259]:
sql = '''
SELECT customer_id
   FROM application
   group by customer_id
   HAVING COUNT(APP_ID) > 2'''
customer2 = pd.read_sql(sql, con)
customer2[:5]

,customer_id
0,9
1,13
2,33
3,42
4,49
